# Project: Analysis of sales and ratings of games

by Nikita Logunov

<font color='black' style='font-size:24px; font-weight:bold'>Project goals and objectives</font>
<br>In this project, we work with the `new_games.csv` dataset, which contains information on sales of games of different genres and platforms, as well as user and critic ratings of games. Our task is to get acquainted with the data, check its correctness and conduct pre-processing, having received the necessary data slice. Then categorize the games by user and critic ratings. And highlight the top-7 platforms by the number of games released over the entire required period.

<font color='black' style='font-size:24px; font-weight:bold'>Description of data</font>
<br>
The `new_games.csv` dataset contains the following fields:

- `Name` — the name of the game.
- `Platform` — the name of the platform.
- `Year of Release` — the year the game was released.
- `Genre` — the genre of the game.
- `NA sales` — sales in North America (in millions of copies sold).
- `EU sales` — sales in Europe (in millions of copies sold).
- `JP sales` — sales in Japan (in millions of copies sold).
- `Other sales` — sales in other countries (in millions of copies sold).
- `Critic Score` — critics' score (from 0 to 100).
- `User Score` — user score (from 0 to 10).
- `Rating` — rating from the ESRB (Entertainment Software Rating Board). This association determines the rating of computer games and assigns them an appropriate age category.

<font color='black' style='font-size:24px; font-weight:bold'>Table of contents</font>
<br>
1. [Loading and studing the data](#1-bullet)
2. [Checking for data errors and preprocessing them](#2-bullet)
   -  [Names, or labels, of dataframe columns](#2.1-bullet)
   -  [Converting data types](#2.2-bullet)
   -  [Handling gaps in data](#2.3-bullet)
   -  [Explicit and implicit duplicates in data](#2.4-bullet)
3. [Filtering data](#3-bullet)
4. [Categorizing data](#4-bullet)
5. [General conclusion](#5-bullet)

<a id="1-bullet"></a>
## Loading and studing the data

Let's load the necessary libraries for data analysis and the data from the `new_games.csv` dataset. Then we'll output the basic information about the data using the `.info()` method and the first rows of the dataframe.

In [7]:
# Import the "pandas" library
import pandas as pd

In [8]:
# Upload new_games.csv to a dataframe
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [9]:
# Displaying information about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


For ease of working with data, column names can be converted to **snake_case** and lowercase.

In [11]:
# Displaying the first lines of the dataframe
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


The `new_games.csv` dataset contains 11 columns and 16956 rows, which contain information about sales of games made in different genres and released on different platforms, as well as user and expert ratings of games.

Let's study the data types and their correctness:
- **Floating-point numeric values (float64).** Four columns: `Year of Release`, `NA sales`, `Other sales`, `Critic Score` - are represented by the `float64` type. `Year of Release` - the year of the game's release is better represented by the `int16` type (it is possible that when filling in the data, there were strings in place of gaps (for example, "no data"), then they were converted using `pd.to_numeric()` with the `errors='coerce'` parameter and got `NaN` instead of strings and floating-point numbers instead of integers). For the other columns, the data type is correct, since ratings and scores are often fractional numbers.
But in the `Critic Score` column, all values have a zero remainder. For ease of perception, it is better to convert this column to an integer type.
- **String data (object).** Seven columns have the `object` data type:
- `Name` and `Genre` contain string information (game name and genre), which is logical for text data. The `object` data type is appropriate here.
- `EU sales`, `JP sales`, `User Score` store information about game sales and user ratings. For such data, it is recommended to use the `float64` type, which is easier to use various aggregation methods with. Perhaps the data is string, since it was obtained from sources other than the data in the `NA sales`, `Other sales`, `Critic Score` columns with the `float64` type.
- `Rating`, `Platform` store text data about rating and platform, but they can be considered as categorical features. In this case, we can use the `category` type to improve performance and optimize memory, since the set of values is limited and known in advance.

After analyzing the data types, it is clear that 7 columns require a data type change.

<a name="2-bullet"></a>
## Checking for data errors and preprocessing them

<a class="anchor" id="2.1-bullet"></a>
### Names, or labels, of dataframe columns

In [15]:
# Let's display the names of all columns in the dataframe and check their writing style
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [16]:
# Let's change all columns to snake_case style
df.columns = df.columns.str.lower().str.replace(' ','_')
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

<a class="anchor" id="2.2-bullet"></a>
### Converting data types

In [18]:
# Saving the number of rows of the original dataframe
df_initial_rows = len(df)

In [19]:
# Display the number of missing rows in a dataframe before converting data types
df.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8714
user_score         6804
rating             6871
dtype: int64

In [20]:
# Calculate the percentage of rows with gaps before converting data types
round(df.isna().sum() / len(df) * 100, 2)

name                0.01
platform            0.00
year_of_release     1.62
genre               0.01
na_sales            0.00
eu_sales            0.00
jp_sales            0.00
other_sales         0.00
critic_score       51.39
user_score         40.13
rating             40.52
dtype: float64

In [21]:
# To convert the data in the df['year_of_release'] column to type 'int16', first remove the gaps in this column
df = df.dropna(subset=['year_of_release'])

In [22]:
# Now we can convert the data in the df['year_of_release'] column to type 'int16'
df['year_of_release'] = df['year_of_release'].astype('int16')

In [23]:
# To convert the data in the df['critic_score'] column to type 'int8', first fill the gaps in this column with a placeholder -1
df['critic_score'] = df['critic_score'].fillna(-1)

In [24]:
# Convert the data in the df['critic_score'] column to type 'int8'
df['critic_score'] = pd.to_numeric(df['critic_score'], errors='coerce', downcast='integer')

In [25]:
# Convert columns df['eu_sales'] and df['jp_sales'] to the smallest 'float' type
# and replace string values of type 'unknown' with NaN
df['eu_sales'] = pd.to_numeric(df['eu_sales'], errors='coerce', downcast='float')
df['jp_sales'] = pd.to_numeric(df['jp_sales'], errors='coerce', downcast='float')

In [26]:
# Convert the df['user_score'] column to the 'float' type of the smallest bit depth
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce', downcast='float')

In [27]:
# Convert columns df[['rating', 'platform']] to type 'category'
df[['rating',  'platform']] = df[['rating', 'platform']].astype('category')

In [28]:
# Checking the result of converting data types
df.dtypes

name                 object
platform           category
year_of_release       int16
genre                object
na_sales            float64
eu_sales            float32
jp_sales            float32
other_sales         float64
critic_score           int8
user_score          float32
rating             category
dtype: object

<a class="anchor" id="2.3-bullet"></a>
### Handling gaps in data

In [30]:
# Display the number of missing rows in a dataframe after converting data types
df.isna().sum()

name                  2
platform              0
year_of_release       0
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score          0
user_score         9123
rating             6780
dtype: int64

In [31]:
# Calculate the percentage of rows with gaps after converting data types
round(df.isna().sum() / len(df) * 100, 2)

name                0.01
platform            0.00
year_of_release     0.00
genre               0.01
na_sales            0.00
eu_sales            0.04
jp_sales            0.02
other_sales         0.00
critic_score        0.00
user_score         54.69
rating             40.65
dtype: float64

There are gaps in the following columns in the data (their number changed after type conversion):
- `name`: 2 rows (0.01% of the data) are missing information about the game name. This is an insignificant share, these gaps can be ignored and removed.
- `genre`: 2 rows (0.01% of the data) are missing information about the game genre. This is also an insignificant share, these gaps can be ignored and removed. It is possible that the gaps in this field are related to the gaps in `Name`, given the overall number of missing rows.
- `year_of_release`: 275 rows (1.62% of the data) are missing information about the year of release of the game. This is an insignificant share of the data, such rows can be removed, which is what we did to correctly convert the data type to integer.
- `eu_sales`: 6 rows (0.04% of the data) are missing information about the number of copies sold in the EU region. These gaps are random, but we can fill them with an average value depending on the platform name and the year of the game's release.
- - `eu_sales`: 4 rows (0.02% of the data) are missing data on the number of copies sold in the JP region. We can also fill these gaps with an average value depending on the platform name and the year of the game's release.
- `critic_score`: 8596 rows (51.53% of the data) are missing critic scores.
- `user_score`: 9123 rows (54.69% of the data) are missing user scores.
- `rating`: 6780 rows (40.65% of the data) are missing ratings.

The last three fields with gaps are most often related and occur in the same row. This may be due to the fact that for the same game this data is only available for one platform (for example, the first platform of the game's release), while ratings and scores were not collected for the other platforms. There is also a possibility that for very old games or for games from other regions that are not translated into English, there are not enough critic and user ratings for natural reasons.

There are a lot of gaps, so we will ignore them. In the `critic_score` column, we will fill the gaps with a "placeholder" -1 to correctly convert the data to an integer type.

In [33]:
# Remove rows with gaps in columns 'name', 'genre'
df.dropna(subset=['name', 'genre'], inplace=True)

In [34]:
# Count the number of deleted lines with gaps
rows_deleted = df_initial_rows - len(df)

In [35]:
# Fill in the gaps in the columns 'eu_sales' and 'jp_sales'. Group the data by platform and year, take the column
# and use the .transform() method to apply an anonymous function (filling the gaps with the average value for the given platform and year)
df.loc[:, 'eu_sales'] = df.groupby(['platform', 'year_of_release'], observed=True)['eu_sales'].transform(lambda x: x.fillna(x.mean()))
df.loc[:, 'jp_sales'] = df.groupby(['platform', 'year_of_release'], observed=True)['jp_sales'].transform(lambda x: x.fillna(x.mean()))

<a class="anchor" id="2.4-bullet"></a>
### Explicit and implicit duplicates in data

We will study unique values in categorical data with the names of the game genre, platform, rating, and year of release. Let's check if there are any hidden duplicates in the data due to typos or different spellings.

In [38]:
# Output unique values in columns 'platform','year_of_release','genre','rating'
for column in ['platform','year_of_release','genre','rating']:
    print(f'Unique values in column {column}:')
    print(df[column].sort_values().unique())
    print()

Unique values in column platform:
['2600', '3DO', '3DS', 'DC', 'DS', ..., 'Wii', 'WiiU', 'X360', 'XB', 'XOne']
Length: 31
Categories (31, object): ['2600', '3DO', '3DS', 'DC', ..., 'WiiU', 'X360', 'XB', 'XOne']

Unique values in column year_of_release:
[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007
 2008 2009 2010 2011 2012 2013 2014 2015 2016]

Unique values in column genre:
['ACTION' 'ADVENTURE' 'Action' 'Adventure' 'FIGHTING' 'Fighting' 'MISC'
 'Misc' 'PLATFORM' 'PUZZLE' 'Platform' 'Puzzle' 'RACING' 'ROLE-PLAYING'
 'Racing' 'Role-Playing' 'SHOOTER' 'SIMULATION' 'SPORTS' 'STRATEGY'
 'Shooter' 'Simulation' 'Sports' 'Strategy']

Unique values in column rating:
['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T', NaN]
Categories (8, object): ['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T']



There are implicit duplicates in the `genre` column in the data. Each value is duplicated in uppercase. It is possible that the data was presented differently in different sources and was not converted to the same form when loaded into the dataframe. Let's convert them to a single form in lowercase and remove the duplicates.

In [40]:
# Number of rows in the dataframe before removing duplicates
initial_row_count = len(df)
f'Number of rows before removing duplicates: {initial_row_count}'

'Number of rows before removing duplicates: 16679'

In [41]:
# Convert the values in the 'genre' column to lowercase
df.loc[:, 'genre'] = df['genre'].str.lower()

In [42]:
# Find the number and proportion of duplicates
count_duplicates = df.duplicated().sum()
duplicates_share = round(count_duplicates / initial_row_count * 100, 2)

In [43]:
f'There were {count_duplicates} duplicate rows found, which is {duplicates_share}% of the total number of rows.'

'There were 235 duplicate rows found, which is 1.41% of the total number of rows.'

In [44]:
#Removing obvious duplicates
df.drop_duplicates(inplace=True)

In [45]:
# Number of rows in the dataframe after removing duplicates
final_row_count = len(df)
f'Number of rows after removing duplicates: {final_row_count}'

'Number of rows after removing duplicates: 16444'

In [46]:
# Convert the values in column 'name' to lowercase (to check for implicit duplicates)
df.loc[:, 'name'] = df['name'].str.lower()

In [47]:
# Check if there are any duplicates left
df.duplicated().sum()

0

In [48]:
f'Total rows removed (gaps and duplicates): {count_duplicates + rows_deleted}'
f'Number of rows in DataFrame decreased by {round((count_duplicates + rows_deleted)*100/df_initial_rows, 2)}%'

'Number of rows in DataFrame decreased by 3.02%'

<font color='black' style='font-size:24px; font-weight:bold'>Conclusion:</font>
<br>The gaps that were decided to be removed were found in the `genre`, `name` and `year_of_release` columns: a total of 277 rows with gaps. The only duplicates in the dataframe were in the `genre` column (the same words, but in uppercase). After eliminating duplicates, 16444 rows remained out of 16679. 235 duplicates were removed (1.41% of the rows). In total, the number of rows in the dataframe was reduced by 512 rows, or 3.02% of the original number.

<a class="anchor" id="3-bullet"></a>
## Filtering data

In [103]:
# Filter the data for 2000-2013 and put it into a new dataframe
df_actual = df[ df['year_of_release'].between(2000,2013) ]

In [105]:
# Output information about the new dataframe
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12781 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             12781 non-null  object  
 1   platform         12781 non-null  category
 2   year_of_release  12781 non-null  int16   
 3   genre            12781 non-null  object  
 4   na_sales         12781 non-null  float64 
 5   eu_sales         12781 non-null  float32 
 6   jp_sales         12781 non-null  float32 
 7   other_sales      12781 non-null  float64 
 8   critic_score     12781 non-null  int8    
 9   user_score       6483 non-null   float32 
 10  rating           8723 non-null   category
dtypes: category(2), float32(3), float64(2), int16(1), int8(1), object(2)
memory usage: 713.1+ KB


In [53]:
# Look at the first lines of the filtered dataframe
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,Wii,2006,sports,41.36,28.959999,3.77,8.45,76,8.0,E
2,mario kart wii,Wii,2008,racing,15.68,12.760000,3.79,3.29,82,8.3,E
3,wii sports resort,Wii,2009,sports,15.61,10.930000,3.28,2.95,80,8.0,E
6,new super mario bros.,DS,2006,platform,11.28,9.140000,6.50,2.88,89,8.5,E
7,wii play,Wii,2006,misc,13.96,9.180000,2.93,2.84,58,6.6,E


<a class="anchor" id="4-bullet"></a>
## Categorizing data


In [113]:
# Divide all games into categories based on user ratings
df_actual.loc[:, 'user_score_group'] = pd.cut(df_actual['user_score'], bins=[0, 3, 8, 10.001], right=False, labels=["Low Rating", "Average Rating", "High Rating"])

In [115]:
# Divide all games by critics' scores into categories
df_actual.loc[:, 'critic_score_group'] = pd.cut(df_actual['critic_score'], bins=[0, 30, 80, 100.001], right=False, labels=["Low score", "Average score", "High score"])

In [57]:
# Look at the first rows of the dataframe with the added categories
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_group,critic_score_group
0,wii sports,Wii,2006,sports,41.36,28.959999,3.77,8.45,76,8.0,E,High Rating,Average score
2,mario kart wii,Wii,2008,racing,15.68,12.760000,3.79,3.29,82,8.3,E,High Rating,High score
3,wii sports resort,Wii,2009,sports,15.61,10.930000,3.28,2.95,80,8.0,E,High Rating,High score
6,new super mario bros.,DS,2006,platform,11.28,9.140000,6.50,2.88,89,8.5,E,High Rating,High score
7,wii play,Wii,2006,misc,13.96,9.180000,2.93,2.84,58,6.6,E,Average Rating,Average score


In [58]:
# Group the data by platforms and calculate the size of each group
count_platform = df_actual.groupby('platform', observed=True).size().reset_index(name='count')

In [59]:
# We sort platforms by the number of games in descending order
platform_counts_sorted = count_platform.sort_values(by='count', ascending=False)

In [60]:
# We list the top-7 platforms for the period from 2000 to 2013
platform_counts_sorted.head(7)

,platform,count
9,PS2,2127
2,DS,2120
15,Wii,1275
12,PSP,1180
17,X360,1121
10,PS3,1087
4,GBA,811


<a class="anchor" id="5-bullet"></a>
## General conclusion:
As a result of data preprocessing, the data frame was cleared of gaps and duplicates, and data types were converted. In total, the number of rows in the data frame was reduced by 512 rows, or 3.02% of the original number.
We were interested in data for 2000-2013, so we sliced the data using a Boolean mask with `.between()` method and saved it to a new dataset, which we continued working with.
Next, we added new columns `user_score_group` and `critic_score_group`, and assigned each game to the `High rating`, `Average rating` and `Low rating` groups - depending on user and critic ratings.
Using grouping and sorting, we found the top-7 platforms by the number of games. The largest platforms from 2000-2013 are `PS2` and `DS`.